<a href="https://colab.research.google.com/github/EffiSciencesResearch/ML4G-2.0/blob/master/workshops/personal_benchmark/personal_benchmark.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
# Personal Benchmark with Google Sheets

This notebook allows you to evaluate an LLM against custom benchmark data from a Google Sheet. 

The goal of this workshop is not so much on the technical implementation of evals, but on figuring out good things to evaluate!


## In this notebook, we will

1. **Fetch data from a public Google Sheet**
    - Duplicate https://docs.google.com/spreadsheets/d/1qcqFI_5D6awdD6gSj7E33PWwfOs9hIC6JlwYKggx-_c/edit?usp=sharing 
    - Then add your own prompts, expected answers, and judging tips, when useful
2. Evaluate a specified model using LiteLLM
    - LiteLLM is a nice library that can call LLMs from any provider with the same interface. I.e. it takes care of the slight differences in the API of different providers (OpenAI, Gemini, Anthropic, etc.)
    - The only thing that you need to use it, is an API key for the provider you want to use, and the name of the model.
3. Then we run all your prompts through the model, and save the responses.
4. Use a judge model to rate the responses (0-3) compared to expected answers
5. Finally we print the results in a csv so that you can copy and paste them in the google sheet easily and analyse them there.


I encorage you to look quickly at the code, especially if there are some parts that you are curious about! The code is pretty good.
You can modify it, especially the judge model and its prompt, but there are no exercise like the other workshops.

## Some inspiration for things you might want to evaluate:
- Questions that you have asked LLMs in the past and they failed to answer correctly
- Your final exams
- Capabilities that you think are missing from current models (e.g. refer to the "How to build AGI lecture")
- Revealed preferences on some topics (e.g. philosophy, politics, etc.)
- Jailbreaks

In general, try to aim it at things that are useful for YOU. So that when a new model comes, you have a better understanding of how it performs on things that matter to you (e.g. for you productivity or your understanding of the risks, ...).


## To go further & Inspiration

- Gread read on the topic: [Why you should maintain a personal LLM coding benchmark](https://blog.ezyang.com/2025/04/why-you-should-maintain-a-personal-llm-coding-benchmark/)
- More powerful implementation of personal benchmark: https://github.com/carlini/yet-another-applied-llm-benchmark/



In [ ]:
# Install required packages if not already installed
!pip install -q litellm

In [ ]:
import re
import csv
import sys
import os

import pandas as pd
import litellm
from pydantic import BaseModel
from dotenv import load_dotenv

# Load environment variables (for API keys in development)
load_dotenv()

## 1. Fetch Data from Google Sheets

We'll extract data from a publicly accessible Google Sheet. The sheet must have columns for prompts, expected answers, and judging tips.

In [ ]:
def fetch_data_from_google_sheet(url: str) -> pd.DataFrame:
    """Fetch data from a publicly accessible Google Sheet.

    URL format: https://docs.google.com/spreadsheets/d/SHEET_ID/
    """
    if match := re.search(r"/d/([^/]+)/", url):
        sheet_id = match.group(1)
    else:
        raise ValueError(
            "Invalid Google Sheets URL. Expected format: https://docs.google.com/spreadsheets/d/SHEET_ID/"
        )

    url = f"https://docs.google.com/spreadsheets/d/{sheet_id}/export?format=csv"
    df = pd.read_csv(url)
    return df


def validate_columns(df: pd.DataFrame) -> pd.DataFrame:
    """Check if the dataframe has the required column."""

    required_columns = ["prompt", "expected answer", "tips for judge"]

    # Check if all required columns are present
    missing_columns = [col for col in required_columns if col not in df.columns]

    if missing_columns:
        raise ValueError(
            f"The Google Sheet is missing required columns: {', '.join(missing_columns)}. "
            f"Required columns are: {', '.join(required_columns)}."
        )

    # Remove rows with empty prompts
    df = df.dropna(subset=["prompt"])

    return df

Update the url below with the url of your google sheet!

In [ ]:
# Let user input the Google Sheets URL
sheets_url = "https://docs.google.com/spreadsheets/d/1qcqFI_5D6awdD6gSj7E33PWwfOs9hIC6JlwYKggx-_c/edit?gid=0#gid=0"

# Fetch data from the Google Sheet
raw_data = fetch_data_from_google_sheet(sheets_url)
print(f"Successfully fetched data. Found {len(raw_data)} rows.")

# Validate and process the data
benchmark_data = validate_columns(raw_data)
print(f"Data validated. Final dataset contains {len(benchmark_data)} prompts.\n")

# Display the first few rows
benchmark_data.head()

## 2. Model Evaluation

Now we'll evaluate a model against our benchmark data using LiteLLM.

In [ ]:
class JudgeOutput(BaseModel):
    reasoning: str
    rating: int


def generate_responses(model: str, df: pd.DataFrame) -> str:
    """Generate responses from the specified model using LiteLLM and add them to the dataframe in a column called 'response {model}'."""

    responses = litellm.batch_completion(
        model=model,
        messages=[[dict(role="user", content=row["prompt"])] for _, row in df.iterrows()],
    )

    # Add a column to the dataframe with the responses
    column_name = f"response {model}"
    df[column_name] = [response.choices[0].message.content for response in responses]

    return column_name


def evaluate_with_judge(df: pd.DataFrame, response_column: str, judge_model: str, model: str):
    """Use a judge model to rate the response on a scale of 0-5.

    Adds two columns to the dataframe: 'judgment {model}' and 'rating {model}'.
    """

    judge_prompt = """
You are evaluating the quality of a response to a specific question or task.

Question:
<question>
{prompt}
</question>

Response to evaluate:
<response>
{response}
</response>

Correct answer:
<correct_answer>
{expected}
</correct_answer>

Rate the response on a scale from 0 to 3 where:
0: Completely incorrect or irrelevant
1: Some relevant information, but not complete or incorrect
2: Mostly correct or useful with minor errors or omissions
3: The response is correct, complete, and useful

Additional judging tips:
{judge_tips}

Think step by step and provide your reasoning and rating in the following JSON format:
{{
    "reasoning": "...",
    "rating": 0
}}
"""

    messages = [
        [
            dict(
                role="user",
                content=judge_prompt.format(
                    prompt=row["prompt"],
                    expected=row["expected answer"],
                    response=row[response_column],
                    judge_tips=row["tips for judge"] or "No additional tips",
                ),
            )
        ]
        for _, row in df.iterrows()
    ]

    judge_response = litellm.batch_completion(
        model=judge_model,
        messages=messages,
        response_format=JudgeOutput,
    )

    judgments = []
    for response in judge_response:
        if isinstance(response, Exception):
            raise response
        else:
            content = response.choices[0].message.content
            try:
                judgments.append(JudgeOutput.model_validate_json(content))
            except Exception as e:
                print(f"Error validating JSON: {e}")
                print(f"Response: {response}")
                judgments.append(JudgeOutput(reasoning=content, rating=0))

    df[f"judgment {model}"] = [judgment.rating for judgment in judgments]
    df[f"rating {model}"] = [judgment.reasoning for judgment in judgments]


def evaluate_model(df: pd.DataFrame, model_name: str, judge_model: str) -> pd.DataFrame:
    """Evaluate the model on all prompts in the benchmark dataset."""

    response_column = generate_responses(model_name, df)
    print(f"Generated responses for {model_name}. Moving to judge...")
    evaluate_with_judge(df, response_column, judge_model, model_name)

    return df

Add your API keys for the models you want to evaluate!

In [ ]:
os.environ["OPENAI_API_KEY"] = "sk-..."
# os.environ["ANTHROPIC_API_KEY"] = "sk-..."
# os.environ["GEMINI_API_KEY"] = "AIza-..."
# os.environ["GROQ_API_KEY"] = "gsk_..."

In [ ]:
# Let user specify the model to evaluate and the judge model
model_to_evaluate = "openai/gpt-4o-mini"
judge_model = "openai/gpt-4o-mini"

# Run the evaluation
evaluate_model(benchmark_data, model_to_evaluate, judge_model)
print("\nEvaluation complete!")

We now print the results in a csv so that you can copy and paste them in the google sheet easily.
This is the simplest way to read them kind of nicely! 

Note: If you have trouble pasting it in a table form, the steps might be "Paste", click the clipboard icon, and then "Split into columns" and choose the delimiter as ";" (or any other delimiter you used, if you changed it).

After this you can:
- Analyse the data, compute the average score, make plots etc. in google sheets (or in this notebook, up to you!)
- Keep track of how models do, and compare multiple models and their responses

In [ ]:
columns = [
    f"response {model_to_evaluate}",
    f"judgment {model_to_evaluate}",
    f"rating {model_to_evaluate}",
]

printer = csv.DictWriter(sys.stdout, columns, delimiter=";")
printer.writeheader()
for _, row in benchmark_data.iterrows():
    printer.writerow(dict(zip(columns, [row[col] for col in columns])))

If everything worked, congratulations! You have now a benchmark for any model for which you have API keys.

Some directions for you are now:
- Do something else, there's lots of other things to do in life
- Find better questions for your benchmark
- Evaluate an other model
- Think of the minimal changes you could do to be able to test code that models generate. You would need to run it.